In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = "2"

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

In [3]:
physical_devices = tf.config.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [4]:
(x_train, y_train),(x_test,y_test)  = mnist.load_data()
x_train = x_train.reshape(-1,28,28,1).astype('float32') / 255.0
x_test = x_test.reshape(-1,28,28,1).astype('float32') / 255.0

In [5]:
model = keras.Sequential(
    [
        layers.Input(shape=(28,28,1)),
        layers.Conv2D(64,3,padding='same'),
        layers.ReLU(),
        layers.Conv2D(128,3, padding='same'),
        layers.ReLU(),
        layers.Flatten(),
        layers.Dense(10,activation='softmax')
    ]
)

In [13]:
class CustomFit(keras.Model):
    def __init__(self,model):
        super(CustomFit, self).__init__()
        self.model = model
    
    #define 1 training step
    #example is GAN where we can customize training loop
    
    def compile(self, optimizer, loss):
        #custom compile
        super(CustomFit, self).compile()
        self.optimizer = optimizer
        self.loss = loss
    
    def train_step(self, data):
        x,y = data
        
        with tf.GradientTape() as tape:
            #now forward prop so tape will record all the grads
            y_pred  = self.model(x, training=True)
            loss = self.loss(y,y_pred)
        
        training_vars = self.trainable_variables
        gradients = tape.gradient(loss, training_vars) #grad of loss wrt traiing vars
        
        self.optimizer.apply_gradients(zip(gradients, training_vars))
        acc_metric.update_state(y,y_pred)
        
        return {'loss':loss, "accuracy":acc_metric.result()}
    
    
    def test_step(self,data):
        #evaluation loop
        x,y = data
        
        y_pred = self.model(x, training=False)
        loss = self.loss(y,y_pred)
        acc_metric.update_state(y,y_pred)
        return {'loss':loss, "accuracy":acc_metric.result()}
        

In [14]:
acc_metric = keras.metrics.SparseCategoricalAccuracy(name='accuracy')

training = CustomFit(model)
training.compile(
    optimizer = keras.optimizers.Adam(),
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
)

training.fit(x_train, y_train, batch_size = 32, epochs = 2)

Epoch 1/2
1875/1875 [==============================] - 8s 4ms/step - loss: 1.4813 - accuracy: 0.9810
Epoch 2/2
1875/1875 [==============================] - 8s 4ms/step - loss: 1.4811 - accuracy: 0.9799


In [15]:
training.evaluate(x_test,y_test)

313/313 [==============================] - 1s 3ms/step - loss: 1.4782 - accuracy: 0.9798


[]